<a href="https://colab.research.google.com/github/aubrin-s/AEOP-REAP/blob/master/HousePrices_aubrin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#House Prices

Predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable

## Open Files

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv('train_house.csv')
test = pd.read_csv('test_house.csv')

train.sort_values(by=['SalePrice'], inplace=True)

train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
495,496,30,C (all),60.0,7879,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,4,5,1920,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,495,Unf,0,225,720,GasA,...,N,FuseA,720,0,0,720,0,0,1,0,2,1,TA,4,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,N,0,523,115,0,0,0,NaN,GdWo,NaN,0,11,2009,WD,Abnorml,34900
916,917,20,C (all),50.0,9000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,2,3,1949,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,TA,CBlock,TA,TA,Av,BLQ,50,Unf,0,430,480,GasA,...,N,FuseA,480,0,0,480,1,0,0,0,1,1,TA,4,Typ,0,NaN,Detchd,1958.0,Unf,1,308,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,10,2006,WD,Abnorml,35311
968,969,50,RM,50.0,5925,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,3,6,1910,1950,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,600,600,Grav,...,N,SBrkr,600,368,0,968,0,0,1,0,2,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Y,0,0,0,0,0,0,NaN,GdWo,NaN,0,5,2009,WD,Abnorml,37900
533,534,20,RL,50.0,5000,Pave,NaN,Reg,Low,AllPub,Inside,Mod,BrkSide,Norm,Norm,1Fam,1Story,1,3,1946,1950,Gable,CompShg,VinylSd,VinylSd,None,0.0,Fa,Fa,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,...,N,FuseF,334,0,0,334,0,0,1,0,1,1,Fa,2,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,N,0,0,0,0,0,0,NaN,NaN,NaN,0,1,2007,WD,Normal,39300
30,31,70,C (all),50.0,8500,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Feedr,Norm,1Fam,2Story,4,4,1920,1950,Gambrel,CompShg,BrkFace,BrkFace,None,0.0,TA,Fa,BrkTil,TA,TA,No,Unf,0,Unf,0,649,649,GasA,...,N,SBrkr,649,668,0,1317,0,0,1,0,3,1,TA,6,Typ,0,NaN,Detchd,1920.0,Unf,1,250,TA,Fa,N,0,54,172,0,0,0,NaN,MnPrv,NaN,0,7,2008,WD,Normal,40000


## Functions

In [ ]:
import pandas  as pd
import numpy as np
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error,median_absolute_error, r2_score
from sklearn.metrics import make_scorer, accuracy_score, f1_score, recall_score, precision_score
from sklearn import svm, tree
from sklearn.model_selection import train_test_split

train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')


# Accuracy
def accuracy(clf,X_val,y_val):
  acc = round(clf.score(X_val, y_val) * 100, 2)
  print ("Validation Accuracy: " + str(acc) + '%')

def acc_score(y_val,y_pred_val):
    acc = round(accuracy_score(y_val, y_pred_val) *100, 2)
    print("Validation accuracy:" + str(acc) + '%')

# Recall
def recall(y_val,y_pred_val):
    recall = round(recall_score(y_val, y_pred_val) *100, 2)
    print("Validation Recall:" + str(recall) + '%')

# Precision
def precision(y_val,y_pred_val):
    precision = round(precision_score(y_val, y_pred_val) *100, 2)
    print("Validation Precision:" + str(precision) + '%')
# F1
def f1(y_val,y_pred_val):
    f1 = round(f1_score(y_val, y_pred_val) *100, 2)
    print("Validation F1:" + str(f1) + '%')


## Mean absolute error
def mae(y_true,y_pred):
  mae = mean_absolute_error(y_true, y_pred)
  print(mae)
## Mean squared error
def mse(y_true,y_pred):
  mse = mean_squared_error(y_true, y_pred)
  print(mse)
## Median absolute error
def mdae(y_true,y_pred):
  mdae = mean_squared_error(y_true, y_pred)
  print(mdae)
## r2 score
def r2(y_true,y_pred):
    r2 = r2_score(y_true,y_pred)
    print(r2*100)

'''
## what i used before:
features = ['OverallQual' , 'GrLivArea' , 'TotalBsmtSF' , 'BsmtFinSF1' ,
            '2ndFlrSF'    , 'GarageArea', '1stFlrSF'    , 'YearBuilt'  ]
X_train       = train[features]
y_train       = train["SalePrice"]
final_X_test  = test[features]

X_train      = X_train.fillna(X_train.mean())
final_X_test = final_X_test.fillna(final_X_test.mean())

## train test split
num_test=0.2
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=num_test, random_state=23)

'''

###From Notebook

train_ID = train['Id']
test_ID = test['Id']
#train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)


ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})

all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

print('Shape all_data: {}'.format(all_data.shape))
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

all_data = pd.get_dummies(all_data)
print(all_data.shape)

train = all_data[:ntrain]
test = all_data[ntrain:]



num_test=0.2
X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=num_test, random_state=23)


all_data size is : (2917, 80)
Shape all_data: (2917, 79)
(2917, 221)


## Gaussain Process Regressor

In [ ]:
##New one
kernel = DotProduct() + WhiteKernel()
gp2 = GaussianProcessRegressor(kernel=kernel)
gp2.fit(X_train, y_train)

y_pred_gp2 = gp2.predict(X_val)


mae(y_val,y_pred_gp2)
mse(y_val,y_pred_gp2)
mdae(y_val,y_pred_gp2)
r2(y_val,y_pred_gp2)
accuracy(gp2,X_val,y_val)

23939.895642803956
1183575270.3651946
1183575270.3651946
80.6042112924104
Validation Accuracy: 80.6%


In [ ]:
##Old One
kernel = DotProduct() + WhiteKernel()
regressor = GaussianProcessRegressor(kernel=kernel)
regressor.fit(X_train, y_train)

y_pred_gp = regressor.predict(X_val)

## mean absolute error
mae(y_val,y_pred_gp)
## mean squared error
mse(y_val,y_pred_gp)
## median absolute error
mdae(y_val,y_pred_gp)
##r2 score
r2(y_val,y_pred_gp)
## acc
accuracy(regressor,X_val,y_val)
## recall
#recall(y_val, y_pred_gp)
## precision
#precision(y_val, y_pred_gp)
## f1
#f1(y_val, y_pred_gp)

23686.630213797907
1020767737.3367517
1020767737.3367517
79.75459937448527
Validation Accuracy: 79.75%


## SVM

In [ ]:
from sklearn import svm
svr = svm.SVR()
svr.fit(X_train, y_train)

y_pred_svr = svr.predict(X_val)

mae(y_val,y_pred_svr)
mse(y_val,y_pred_svr)
mdae(y_val,y_pred_svr)
r2(y_val,y_pred_svr)

accuracy(svr,X_val,y_val)

#recall(y_val, y_pred)

#precision(y_val, y_pred)

#f1(y_val, y_pred)

55952.984627400605
6412212143.419724
6412212143.419724
-5.079850007033837
Validation Accuracy: -5.08%


##Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = tree.DecisionTreeRegressor()
clf.fit(X_train, y_train)

y_pred_dt = clf.predict(X_val)

mae(y_val,y_pred_dt)
mse(y_val,y_pred_dt)
mdae(y_val,y_pred_dt)
r2(y_val,y_pred_dt)

accuracy(clf,X_val,y_val)

#recall(y_val, y_pred)

#precision(y_val, y_pred)

#f1(y_val, y_pred)

28214.561643835616
1587611224.3835616
1587611224.3835616
73.98308951788091
Validation Accuracy: 73.98%


##Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred_rf = clf.predict(X_val)

mae(y_val,y_pred_rf)
mse(y_val,y_pred_rf)
mdae(y_val,y_pred_rf)
r2(y_val,y_pred_rf)

accuracy(clf,X_val,y_val)

23281.013698630137
1136805725.869863
1136805725.869863
81.3706451861346
Validation Accuracy: 2.4%


##K Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
KNeighborsClassifier()
y_pred_knn = knn.predict(X_val)

mae(y_val,y_pred_knn)
mse(y_val,y_pred_knn)
mdae(y_val,y_pred_knn)
r2(y_val,y_pred_knn)

accuracy(knn,X_val,y_val)


35416.09589041096
2345098324.1369863
2345098324.1369863
53.48848387170994
Validation Accuracy: 1.71%


## Nearest Centroid Classifier

In [ ]:
from sklearn.neighbors import NearestCentroid
nc = NearestCentroid()
nc.fit(X_train, y_train)
NearestCentroid()
y_pred_nc= nc.predict(X_val)

mae(y_val,y_pred_nc)
mse(y_val,y_pred_nc)
mdae(y_val,y_pred_nc)
r2(y_val,y_pred_nc)

accuracy(nc,X_val,y_val)

31178.154109589042
2231968466.75
2231968466.75
55.73224530904002
Validation Accuracy: 0.34%


## Creating submissions

In [ ]:
import pandas as pd

y_pred_test = gp2.predict(test.fillna(0))
submission = pd.DataFrame({"Id": test['Id'], "SalePrice": y_pred_test})
submission.to_csv('housepricesx.csv', index=False)

In [ ]:
## TEST Submission (old one)
predictions = regressor.predict(final_X_test)

output = pd.DataFrame({"Id":test.Id, "SalePrice":predictions})
output.to_csv('houseprices.csv', index=False)
result = pd.read_csv('houseprices.csv')
result